In [96]:
import numpy as np
import photoncount
import matplotlib.pyplot as pl
from ipywidgets import VBox, FloatSlider, FloatRangeSlider, HBox, BoundedFloatText, RadioButtons, Dropdown
import bqplot.pyplot as plt

In [97]:
wid_D = BoundedFloatText(
    value=4.0,
    min=0.01,
    max=8.0,
    step=0.1,
    description='D [m]')

### WAVELENGTH PARAMETERS ###

cwl = Dropdown(
    options=[('Ca II K', 393.33), ('H I beta', 486.1), ('Mg I b', 517.30), ('Fe I', 525.0 ), ('Fe I V', 630.2 ), ('H I alpha', 656.3 ), 
             ('K I', 769.9), ('Ca II', 854.2), ('He I',1083.0), ('Fe I IR', 1564.8)],
    value = 854.2,
    description = 'target wavelength',
    disabled = False
    )

# Note that Fe I @1564 nm will lead to error as there is no data beyon 1100 nm in atmostrans.dat

bp = BoundedFloatText(value=0.5,
    min=0.02,
    max=10.00,
    step=0.01,
    description='bandpass')

cwl_shift = BoundedFloatText(value=0.0,
    min=0.0,
    max=1.00,
    step=0.01,
    description='cwl shift')

wave = BoundedFloatText(value=cwl.value, min= 380.00, step=0.01,                        
    description='wavelength:')

h_wave_widgets = HBox([cwl, wave, cwl_shift, bp])

### TRANSMISSION PARAMETERS ###

wid_T_T = BoundedFloatText(
    value=0.4,
    min=0.0,    
    max=1.0,
    step=0.01,
    description='Telescope transmission')

wid_T_I = BoundedFloatText(
    value=0.3,
    min=0.0,    
    max=1.0,
    step=0.01,
    description='Instrument transmission')

wid_T_D = BoundedFloatText(
    value=0.1,
    min=0.0,    
    max=1.0,
    step=0.01,
    description='Detector QE')

wid_T = BoundedFloatText(
    value=wid_T_T.value * wid_T_I.value * wid_T_D.value,
    min=0.0,    
    max=1.0,
    step=0.01,
    description='Total transmission:')

h_T_widgets = HBox([wid_T_T,wid_T_I, wid_T_D, wid_T])

### SCIENCE REQUIREMENTS PARAMETERS ####

wid_R = BoundedFloatText(
    value=80000.0,
    min=0.0,
    max=1e6,
    step=0.1,
    description='Spectral resolution:')

wid_SN = BoundedFloatText(
    value=1000.0,
    min=0.0,
    max=100000,
    description='Desired S/N:')

wid_v = BoundedFloatText(
    value=7.0,
    min=0.0,
    max=200,
    description='Evolution speed [km/s]:')

wid_pol = RadioButtons(options=['No', 'Yes'], description='Polarization')

### OTHER PARAMETERS ###


wid_bin = RadioButtons(options=['1x1', '2x2', '4x4'], description='Binning')

widgets = VBox([wid_D, h_wave_widgets, wid_pol, wid_bin, wid_R, h_T_widgets, wid_SN, wid_v])


TraitError: Setting min > max

In [ ]:
ph = photoncount.Photocount()

properties_dict = {}
properties_dict['D'] = 4.0
properties_dict['lmin'] = cwl.value + cwl_shift.value - (bp.value/2.0)
properties_dict['lmax'] = cwl.value + cwl_shift.value + (bp.value/2.0)
properties_dict['polarimetry'] = 0
properties_dict['R'] = 8e4
properties_dict['T'] = 0.1
properties_dict['SN'] = 1e3
properties_dict['v'] = 7.0
properties_dict['binning'] = 1.0

ph.set_properties(properties_dict)
ph.compute()

In [ ]:
fig1 = plt.figure(title='Spectrum', legend_location='bottom-right')
fig1.layout.width = '600px'
fig1.layout.height = '400px'
line1a = plt.plot(ph.ll * ph.m_to_nm, ph.Ilambda, 'b', stroke_width=3, figure=fig1, labels=["Atlas"])
line1b = plt.plot(ph.ll * ph.m_to_nm, ph.Ilambdac, 'r', stroke_width=3, figure=fig1, labels=["Degraded"])
plt.legend()

fig2 = plt.figure(title='Optimal pixel size')
fig2.layout.width = '600px'
fig2.layout.height = '400px'
line2 = plt.plot(ph.ll * ph.m_to_nm, ph.dx, 'b', stroke_width=3, figure=fig2)

fig3 = plt.figure(title='Integration time')
fig3.layout.width = '600px'
fig3.layout.height = '400px'
line3a = plt.plot(ph.ll * ph.m_to_nm, ph.t, 'b', stroke_width=3, figure=fig3)
line3b = plt.plot(ph.ll * ph.m_to_nm, ph.tideal, 'r', stroke_width=3, figure=fig3)

fig4 = plt.figure(title='Optimal integration time')
fig4.layout.width = '600px'
fig4.layout.height = '400px'
line4 = plt.plot(ph.ll * ph.m_to_nm, ph.dt, 'b', stroke_width=3, figure=fig4)

top = HBox([fig1, fig2])
bottom = HBox([fig3, fig4])
figs = VBox([top, bottom])

In [ ]:
def update(change): 
    wave.value = cwl.value 
    wid_T.value = wid_T_T.value * wid_T_I.value * wid_T_D.value
    
    properties_dict['D'] = wid_D.value    
    properties_dict['lmin'] = cwl.value + cwl_shift.value - (bp.value/2.0)
    properties_dict['lmax'] = cwl.value + cwl_shift.value + (bp.value/2.0)
    properties_dict['R'] = wid_R.value
    properties_dict['T'] = wid_T.value
    properties_dict['SN'] = wid_SN.value
    properties_dict['v'] = wid_v.value         
        
    if (wid_pol.value == 'Yes'):
        properties_dict['polarimetry'] = 1.0
    else:
        properties_dict['polarimetry'] = 0.0
    
    if (wid_bin.value == '1x1'):
        properties_dict['binning'] = 1.0
        
    if (wid_bin.value == '2x2'):
        properties_dict['binning'] = 2.0
        
    if (wid_bin.value == '4x4'):
        properties_dict['binning'] = 4.0
                    
    ph.set_properties(properties_dict)
    ph.compute()
    
    line1a.x = ph.ll * ph.m_to_nm
    line1a.y = ph.Ilambda
    line1b.x = ph.ll * ph.m_to_nm
    line1b.y = ph.Ilambdac
    
    line2.x = ph.ll * ph.m_to_nm
    line2.y = ph.dx
    
    line3a.x = ph.ll * ph.m_to_nm
    line3a.y = ph.t
    line3b.x = ph.ll * ph.m_to_nm
    line3b.y = ph.tideal
    
    line4.x = ph.ll * ph.m_to_nm
    line4.y = ph.dt
    
# register the above callback with the 'value' trait of the sliders and dropdown menu
for slider in [wid_D, wid_R, wid_T, wid_T_T, wid_T_I, wid_T_D,
               wid_SN, wid_v, wid_pol, wid_bin, bp,  cwl_shift]:
    slider.observe(update, 'value')
cwl.observe(update,'value')

In [98]:
VBox([widgets, figs])